# Backpropagation Complete Example

Adapted from Udacity.

## Task: Imports

In [0]:
import numpy as np
import pandas as pd

## Read and explore the data

In [0]:
url="https://stats.idre.ucla.edu/stat/data/binary.csv"
admissions=pd.read_csv(url)

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing

sample = np.random.choice(data.index, size=int(len(data)*0.8), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


## Activation

In [0]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

## Hyper-parameters

In [0]:
n_hidden = 3  # number of hidden units
epochs = 1000
learnrate = 0.05

## Initialise

In [0]:
n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)

In [0]:
weights_input_hidden

array([[-0.88414475, -0.49050136, -0.46947263],
       [-0.2318007 ,  0.0900469 ,  0.5599944 ],
       [-0.69354118, -0.54641469,  0.42496867],
       [-0.24722968, -0.12128233, -0.00486424],
       [-0.48868866, -0.29390022,  0.33031747],
       [-0.33760577, -1.44261469, -0.02199874]])

In [0]:
weights_hidden_output

array([ 0.42940786, -0.35934767, -0.58729263])

## Epochs

In [0]:
# Iterate over the epochs
for e in range(epochs):
  
    # Initialise the weights
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    
    # Iterate over each training example
    for x, y in zip(features.values, targets):
        
        # *******************
        ## Forward pass ##
        # *******************
        
        hidden_input = np.dot(x,weights_input_hidden)
        hidden_activations = sigmoid(hidden_input)
        output = sigmoid(np.dot(hidden_activations,weights_hidden_output))

        # *******************
        ## Backward pass ##
        # *******************
        
        # The network error on this particular example
        error = y - output

        # Output error
        output_error = error * output * (1-output)

        # propagate errors to hidden layer
        hidden_error = weights_hidden_output * output_error * hidden_activations * (1-hidden_activations) 

        # Update the change in weights
        del_w_hidden_output += output_error * hidden_activations
        del_w_input_hidden += hidden_error * x[:,None]

    # Actual weight updates. Dividing by n_records to deal with
    # potentially large gradient steps
    weights_input_hidden += (learnrate * del_w_input_hidden) / n_features
    weights_hidden_output += (learnrate * del_w_hidden_output) / n_features

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_activations = sigmoid(np.dot(x, weights_input_hidden))
        
        # Get the network output
        out = sigmoid(np.dot(hidden_activations,
                             weights_hidden_output))
        
        # Compute the loss
        loss = np.mean((out - targets) ** 2)

        # Check the current loss with the previous one
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

Train loss:  0.2289558383742399
Train loss:  0.22839068774226695
Train loss:  0.23110554350118218   WARNING - Loss Increasing
Train loss:  0.23201962437407805   WARNING - Loss Increasing
Train loss:  0.23212579813149015   WARNING - Loss Increasing
Train loss:  0.23180531416973188
Train loss:  0.23127915827447715
Train loss:  0.2306840162638036
Train loss:  0.23009183384847165
Train loss:  0.2295275856401587


## Calculate accuracy on test data

In [0]:
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
out

array([0.49544081, 0.35382272, 0.36894177, 0.37133832, 0.14252752,
       0.33309823, 0.16692571, 0.1662274 , 0.33771036, 0.39052442,
       0.4038268 , 0.22896877, 0.203722  , 0.49132056, 0.27142611,
       0.15099789, 0.47927258, 0.40819385, 0.3775395 , 0.3560546 ,
       0.22606235, 0.41216408, 0.24625831, 0.43054272, 0.15741595,
       0.39066986, 0.18744467, 0.38571895, 0.185205  , 0.34512464,
       0.38050809, 0.48869549, 0.24232709, 0.23769005, 0.40649368,
       0.3825608 , 0.29023314, 0.47360245, 0.23762784, 0.40649368,
       0.18438877, 0.30821819, 0.31603477, 0.20085038, 0.16232723,
       0.4410694 , 0.33946466, 0.31640883, 0.16891072, 0.29271125,
       0.42060749, 0.48144947, 0.21489889, 0.31566286, 0.3252848 ,
       0.2814939 , 0.30148505, 0.46266179, 0.1758686 , 0.32436153,
       0.25110489, 0.38200304, 0.27301705, 0.35573648, 0.15595686,
       0.1428985 , 0.49023039, 0.15372522, 0.35371118, 0.45624395,
       0.4427296 , 0.32233651, 0.48594057, 0.42002928, 0.30854

## Predict

In [0]:
predictions = out > 0.5
predictions

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

## Compute accuracy

In [0]:
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Prediction accuracy: 0.713
